In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import pathlib

train_images_dir = pathlib.Path("./archive/Human Bone Fractures Multi-modal Image Dataset (HBFMID)/Bone Fractures Detection") / "train" / "images"
train_labes_dir = pathlib.Path("./archive/Human Bone Fractures Multi-modal Image Dataset (HBFMID)/Bone Fractures Detection") / "train" / "labels"
train_images = list(train_images_dir.glob("*.jpg"))

class_names = ['Comminuted', 'Greenstick', 'Healthy', 'Linear', 'Oblique Displaced', 'Oblique', 'Segmental', 'Spiral', 'Transverse Displaced', 'Transverse']

fig, axes = plt.subplots(3, 3, figsize=(10, 10))
for i, ax in enumerate(axes.flat):
    img = Image.open(train_images[i])
    ax.imshow(img)
    ax.set_title(img.size)
    # ax.axis("off")

    label_path = train_labes_dir / f"{train_images[i].stem}.txt"
    if label_path.exists():
        with open(label_path, "r") as f:
            lines = f.readlines()
        
        img_width, img_height = img.size
        for line in lines:
            values = line.strip().split()
            class_id = int(values[0])
            x_center, y_center, width, height = map(float, values[1:])

            x1 = int((x_center - width / 2) * img_width)
            y1 = int((y_center - height / 2) * img_height)

            box_width = int(width * img_width)
            box_height = int(height * img_height)

            rect = patches.Rectangle((x1, y1), box_width, box_height, linewidth=2, edgecolor="r", facecolor="none")
            ax.add_patch(rect)
            ax.text(x1-20, y1-20, class_names[class_id], color="r", fontsize=12)

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.version.cuda}")  # Should match your installed CUDA (12.x)
print(f"CUDA available: {torch.cuda.is_available()}")  # Must be True

In [2]:
from ultralytics import YOLO
import torch

# Check GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
print(f"GPU name: {torch.cuda.get_device_name(0) if device == 'cuda' else 'CPU'}")  # Show GPU info

# Load YOLO model
try:
    model = YOLO("./yolo11m.pt").to(device)
except:
    print("Custom weights not found, falling back to pretrained YOLOv8")
    model = YOLO("yolov8m.pt").to(device)  # Fallback to standard YOLOv8 medium

# Training parameters
config = {
    "data": "./archive/Human Bone Fractures Multi-modal Image Dataset (HBFMID)/Bone Fractures Detection/data.yaml",
    "epochs": 50,  # Increased from 20 as medical imaging often benefits from more epochs
    "imgsz": 640,
    "batch": 16 if device == 'cuda' else 4,  # Smaller batch for CPU
    "device": device,
    "project": "yolo_fracture_detection",
    "name": "exp1",
    "exist_ok": True,
    "optimizer": "AdamW",  # More modern optimizer
    "lr0": 0.001,  # Lower initial LR for fine-tuning
    "pretrained": True,
    "cos_lr": True,  # Cosine learning rate scheduler
    "weight_decay": 0.0005,  # Regularization
    "fliplr": 0.5,  # Horizontal flip augmentation
    "mosaic": 1.0,  # Mosaic augmentation
    "mixup": 0.2,  # Mixup augmentation
    "close_mosaic": 10,  # Disable mosaic last epochs
    "patience": 15,  # Early stopping patience
    "save_period": 5,  # Save checkpoint every 5 epochs
    "single_cls": False,  # Set True if you have only fracture/non-fracture
}

# Train the model
try:
    results = model.train(**config)
    print("Training completed successfully!")
except Exception as e:
    print(f"Training failed: {str(e)}")

Using device: cuda
GPU name: NVIDIA GeForce RTX 4070 SUPER
Ultralytics 8.3.159  Python-3.13.5 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=./archive/Human Bone Fractures Multi-modal Image Dataset (HBFMID)/Bone Fractures Detection/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=./yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp1,

train: Scanning D:\Coding\CV\BoneClassifier\archive\Human Bone Fractures Multi-modal Image Dataset (HBFMID)\Bone Fractures Detection\train\labels.cache... 1347 images, 3 backgrounds, 0 corrupt: 100%|██████████| 1347/1347 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.30.2 ms, read: 5.31.9 MB/s, size: 25.1 KB)


val: Scanning D:\Coding\CV\BoneClassifier\archive\Human Bone Fractures Multi-modal Image Dataset (HBFMID)\Bone Fractures Detection\valid\labels.cache... 128 images, 0 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]


Plotting labels to yolo_fracture_detection\exp1\labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolo_fracture_detection\exp1
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      8.25G      1.207     0.7492      1.664          3        640: 100%|██████████| 85/85 [00:20<00:00,  4.24it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.03it/s]

                   all        128        157      0.814       0.78      0.843       0.42



50 epochs completed in 0.337 hours.
Optimizer stripped from yolo_fracture_detection\exp1\weights\last.pt, 40.5MB
Optimizer stripped from yolo_fracture_detection\exp1\weights\best.pt, 40.5MB

Validating yolo_fracture_detection\exp1\weights\best.pt...
Ultralytics 8.3.159  Python-3.13.5 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
YOLO11m summary (fused): 125 layers, 20,037,742 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.67it/s]


                   all        128        157      0.814       0.78      0.843      0.422
            Comminuted         15         15       0.91      0.733      0.834      0.481
            Greenstick         10         10      0.754        0.5      0.571      0.192
               Healthy          7          7      0.777          1      0.944      0.646
                Linear          1          1      0.706          1      0.995      0.438
     Oblique Displaced         30         30      0.937      0.867      0.897      0.428
               Oblique          7          7      0.804      0.857      0.924      0.521
             Segmental          3          3      0.466      0.333      0.454      0.303
                Spiral          5          5          1      0.693      0.995      0.371
  Transverse Displaced         53         65       0.93      0.821      0.872      0.357
            Transverse         11         14      0.861          1      0.944      0.481
Speed: 0.2ms preproce

In [ ]:
import torch
print(torch.__version__)             # Should be 2.3.0+
print(torch.cuda.is_available())    # Should return True
print(torch.version.cuda)           # Should show 12.1 (but works with 12.9)

In [1]:
import torchvision
import torch
from torchvision.ops import nms

# Dummy data
boxes = torch.tensor([[0, 0, 10, 10], [0, 0, 10, 10]], dtype=torch.float32).cuda()
scores = torch.tensor([0.9, 0.8], dtype=torch.float32).cuda()

# Prova a chiamare NMS
output = nms(boxes, scores, 0.5)
print(output)

tensor([0], device='cuda:0')
